In [9]:
from PyPDF2 import PdfReader
import os
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chat_models import ChatOpenAI

In [2]:
def extract_text_from_pdf(url: str) -> str:
    reader = PdfReader(url)

    text = ''
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        text += page.extract_text()
    return text

In [5]:
base_path = '../../data/raw/dataset/'
content = []
instruction = []
output = []
with os.scandir(base_path) as entries:
    for entry in entries:
        if entry.name.endswith(".pdf"):
            text = extract_text_from_pdf(base_path+entry.name)
            components = text.split(":")

            name = components[1].replace('Name', '').replace('Description','')#Name and Description
            description = components[2].replace('Data', '')# Description and Data
            data = text.split(":")[3].replace('In-depth Analysis', '').replace('-', '')
            analysis = text.split(":")[4] # In-depth Analysis
            text = name + ' '+ description + ' '+ data + ' '+analysis
            content.append(text.replace('\n', ''))
        elif entry.name.endswith(".txt"):
            id = lambda x: x
            file = open(base_path+entry.name,"r+", encoding="utf8")
            lines = file.readlines()
            text = ' '.join(lines)
            content.append(text)
            file.close()

dataset = {'prompt': content}

In [8]:
with open('../../data/processed/context.txt', 'w', encoding='utf8') as f:
    for c in content:
        f.write(c)
    
f.close()

In [10]:
os.environ['OPENAI_API_KEY'] = key

In [11]:
loader = TextLoader('../../data/processed/context.txt', encoding='utf8')

index = VectorstoreIndexCreator().from_loaders([loader])

In [12]:
print(index.query('What is the Xalaxians collective consciousness?', llm=ChatOpenAI()))

The Xalaxian collective consciousness is a phenomenon in which the individual consciousness of Xalaxians is merged together to form a singular, unified awareness. This allows the species to think, communicate, and make decisions as one collective entity. Through their energy-based forms, Xalaxians are able to exchange information on a quantum level, sharing knowledge, perspectives, and experiences. The collective consciousness is seen as a source of strength, creativity, and support for the Xalaxian people, enabling them to work together to create a better future for their species and the universe as a whole.


In [13]:
print(index.query('do you know about the Xalaxians advanced technology?', llm=ChatOpenAI()))

Yes, I have information about the Xalaxians' advanced technology. They possess highly advanced technology that allows them to manipulate space-time and energy. This enables them to travel vast cosmic distances instantly and explore different realms of existence. They also have advanced communication technology, including telepathy, that allows them to stay connected across galaxies. Additionally, the Xalaxians utilize advanced artificial intelligence to aid in research and problem-solving. Their technology is based on energy manipulation and they have mastered quantum computing. They have also developed advanced medical technology, energy production, and more. The Xalaxians are committed to sharing their knowledge and technology with other civilizations.
